## 河川横断面の簡略化について

### モチベーション

 - 横断面平均一次元河川流計算で実河川を対象とする場合、地形条件は河川横断測量データを使用することが一般的である。
 - このデータより、水位に対する河積、川幅、潤辺等を計算するが計算負荷が大きく、特に長期計算を行う場合には計算速度に大きく影響する。
 - この改善方法として、河川横断測量データの簡略化とその影響について検証する。

### 簡素化

 - 基本の横断図は過去記事のものを使用する。
 - 横断面の簡略化には、pythonのモジュール[shapely](https://shapely.readthedocs.io/en/latest/manual.html)の[simplifyメソッド](https://shapely.readthedocs.io/en/latest/manual.html#object.simplify)を使います。

In [139]:
import numpy as np
from shapely.geometry import LineString, MultiPoint
import json

In [140]:
from IPython.display import IFrame

In [141]:
with open('sample-RiverSection.geojson','r') as f:
    j = json.load(f)

In [142]:
p3d = j['features'][0]['geometry']['coordinates']

In [143]:
def from3Dto2D(pointz, porg=None):
    point = pointz[:,:2]
    if porg is None : porg = pointz[0]
        
    v = point[-1] - point[0]
    e = v/np.linalg.norm(v)
    d = point - porg[:2]
    L = np.dot(d, e)
    Z = pointz[:,2]
    return L, Z

In [144]:
L, Z = from3Dto2D(np.array(p3d))

In [145]:
L = L[5:-6]
Z = Z[5:-6]

In [146]:
line = LineString(np.c_[L,Z])
g = hv.Curve(line.coords[:], label='original').options(width=600, color='k', line_width=4)
print( len(line.coords[:]) )
fig = [g]

for tol in [0.2, 0.5, 1.0, 2.0]:
    line2 = line.simplify(tol, preserve_topology=True)
    g = hv.Curve(line2.coords[:], label='tolerance=' + str(tol))
    print( len(line2.coords[:]) )
    fig.append(g)

80
24
19
12
7


|  |  number of points  |
| ---- | ---- |
|  original  |  80  |
|  tolerance = 0.2  |  24  |
|  tolerance = 0.5  |  19  |
|  tolerance = 1.0  |  12  |
|  tolerance = 2.0  |  7  |

In [147]:
g = hv.Overlay(fig).options(legend_position='top')
hv.save(g, 'test.html')

In [148]:
IFrame("test.html",width=700,height=300)

### 物理量への影響
 - 河川流の場合は、水位$H$と流量$Q$の関係が重要となるのでこれらへの影響を確認する。
 - 等流を仮定して以下のマニングの式によりそれらを評価する。

$$
Q = \frac{1}{n}i_e^{1/2}R^{2/3}A
$$

 - 右辺の径深$R$、河積$A$は水位$H$の関数になるので、$H \sim R^{2/3}A$を比較する。
 - Hが17～24の範囲で図化すると次のとおりとなる。

In [111]:
import riversection as sect
from numba.typed import List

In [149]:
line = LineString(np.c_[L,Z])
p = np.array( line.coords[:] )
s = sect.section(np.array([p]),np.array([np.repeat(0.1, len(p) - 1)]))

hh = np.arange(17,24.01,0.2)
RA = []
for hhh in hh:
    A, B, S = s.H2ABS(hhh)
    RA.append( A**(5/3)/S**(2/3) )

g = hv.Curve((RA, hh), label='original').options(width=600, color='k', line_width=4, xlabel='R^(2/3)A', ylabel='H')
fig = [g]

In [150]:
for tol in [0.2, 0.5, 1.0, 2.0]:
    line2 = line.simplify(tol, preserve_topology=True)
    
    p = np.array( line2.coords[:] )
    s = sect.section(np.array([p]),np.array([np.repeat(0.1, len(p) - 1)]))
    
    hh = np.arange(17,24.01,0.2)
    RA = []
    for hhh in hh:
        A, B, S = s.H2ABS(hhh)
        RA.append( A**(5/3)/S**(2/3) )
    
    g = hv.Curve((RA, hh), label='tolerance=' + str(tol))
    fig.append(g)

In [155]:
g = hv.Overlay(fig).options(legend_position='top')
hv.save(g, 'RA.html')

In [156]:
IFrame("RA.html",width=700,height=300)

### 計算時間への影響

 - 上の$H \sim R^{2/3}A$関係の計算速度を比較しました。
 - 時間計測はjupyterの%%timeitを使ってます。
 - PCのスペック
 - 計算時間は次のとおりで測点数の減少に比例して高速化する。

|  |  computational time |
| ---- | ---- |
|  original  |  686 µs ± 11.5 µs  |
|  tolerance = 0.2  |  288 µs ± 5.87  |
|  tolerance = 0.5  |  265 µs ± 12.1 µs  |
|  tolerance = 1.0  |  207 µs ± 7.21 µs  |
|  tolerance = 2.0  |  174 µs ± 2.27 µs  |

In [158]:
line = LineString(np.c_[L,Z])

In [159]:
%%timeit
p = np.array( line.coords[:] )
s = sect.section(np.array([p]),np.array([np.repeat(0.1, len(p) - 1)]))

hh = np.arange(17,24.01,0.2)
RA = []
for hhh in hh:
    A, B, S = s.H2ABS(hhh)
    RA.append( A**(5/3)/S**(2/3) )

686 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [160]:
tol = 0.2
line2 = line.simplify(tol, preserve_topology=True)

In [162]:
%%timeit
p = np.array( line2.coords[:] )
s = sect.section(np.array([p]),np.array([np.repeat(0.1, len(p) - 1)]))

hh = np.arange(17,24.01,0.2)
RA = []
for hhh in hh:
    A, B, S = s.H2ABS(hhh)
    RA.append( A**(5/3)/S**(2/3) )

288 µs ± 5.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [163]:
tol = 0.5
line2 = line.simplify(tol, preserve_topology=True)

In [164]:
%%timeit
p = np.array( line2.coords[:] )
s = sect.section(np.array([p]),np.array([np.repeat(0.1, len(p) - 1)]))

hh = np.arange(17,24.01,0.2)
RA = []
for hhh in hh:
    A, B, S = s.H2ABS(hhh)
    RA.append( A**(5/3)/S**(2/3) )

265 µs ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [165]:
tol = 1.0
line2 = line.simplify(tol, preserve_topology=True)

In [166]:
%%timeit
p = np.array( line2.coords[:] )
s = sect.section(np.array([p]),np.array([np.repeat(0.1, len(p) - 1)]))

hh = np.arange(17,24.01,0.2)
RA = []
for hhh in hh:
    A, B, S = s.H2ABS(hhh)
    RA.append( A**(5/3)/S**(2/3) )

207 µs ± 7.21 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [167]:
tol = 2.0
line2 = line.simplify(tol, preserve_topology=True)

In [168]:
%%timeit
p = np.array( line2.coords[:] )
s = sect.section(np.array([p]),np.array([np.repeat(0.1, len(p) - 1)]))

hh = np.arange(17,24.01,0.2)
RA = []
for hhh in hh:
    A, B, S = s.H2ABS(hhh)
    RA.append( A**(5/3)/S**(2/3) )

174 µs ± 2.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
